In [4]:
import json
import pandas as pd
import numpy as np
import ijson
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
import cohere
import requests
import datetime
from tqdm import tqdm
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anushmutyala/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
result = pd.read_csv('final_embedding.csv')
result

/Users/anushmutyala/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,name,abstract,citations,publication value,year,authors
0,7C7CAEED,On rank correlation in information retrieval e...,Some methods for rank correlation in evaluatio...,"""""10.1109/ICPR48806.2021.9411963/, """"10.1109...",ACM SIGIR ForumVolume 41Issue 1June 2007 pp ...,NaN,Massimo Melucci
1,7AEE29E3,The Voting Model for People Search,NaN,NaN,NaN,NaN,NaN
2,7D68490B,Document clustering with committees,Document clustering is useful in many informat...,"""""10.1109/ACIT53391.2021.9677217/, """"10.4018...",SIGIR '02: Proceedings of the 25th annual inte...,NaN,Patrick Pantel
3,7A488256,A comparison of indexing techniques for Japane...,A series of Japanese full-text retrieval exper...,"""""10.1109/SNLP.2009.5340946/, """"10.1007/s105...",SIGIR '93: Proceedings of the 16th annual inte...,NaN,Hideo Fujii
4,7D5CD2DF,Feature selection for ranking,Ranking is a very important topic in informati...,"""""10.1007/978-3-030-95391-1_29/, """"10.1007/9...",SIGIR '07: Proceedings of the 30th annual inte...,NaN,Xiubo Geng
...,...,...,...,...,...,...,...
281237,629806,Effectiveness and usability of an online help ...,An empirical study is presented which aims at ...,"8543', '327540', '395578', '397153', '398612",Proceedings of the 10th international conferen...,2008.0,"Jérôme Simonin,Noëlle Carbonell,Danielle Pelé"
281238,629807,Busy period analysis of finite QBD processes,We present the number of customers served and ...,340965,ACM SIGMETRICS Performance Evaluation Review,2008.0,"Chaitanya Garikiparthi,Appie van de Liefvoort,..."
281239,629808,The Grid as a Single Entity: Towards a Behavio...,Grids emerged in the last decade as large dist...,0,Proceedings of the OTM 2008 Confederated Inter...,2008.0,"Jesús Montes,Alberto Sánchez,Julio J. Valdés,M..."
281240,629811,Multimodal system evaluation using modality ef...,"In this paper, we propose two new objective me...","294663', '302639', '572828",Proceedings of the 10th international conferen...,2008.0,"Manolis Perakakis,Alexandros Potamianos"


In [3]:
result_abstracts = result['abstract'].dropna()
result_abstracts

0         Some methods for rank correlation in evaluatio...
2         Document clustering is useful in many informat...
3         A series of Japanese full-text retrieval exper...
4         Ranking is a very important topic in informati...
5         Content-targeted advertising, the task of auto...
                                ...                        
281237    An empirical study is presented which aims at ...
281238    We present the number of customers served and ...
281239    Grids emerged in the last decade as large dist...
281240    In this paper, we propose two new objective me...
281241    In this paper we address the problem of estima...
Name: abstract, Length: 281212, dtype: object

In [15]:
abstract_list = result_abstracts.to_list()
truth_examples = [("clustering algorithms", abstract_list[0]), ("ranking models", abstract_list[1]), ("advertisement optimization", abstract_list[2]), ("social tagging", abstract_list[3]), ("clustering algorithms", abstract_list[4]), ("collaborative filtering", abstract_list[5]), ("sentence ranking", abstract_list[6])]


In [5]:
# Paste your API key here. Remember to not share publicly
api_key = 'S35VtqP4EcUmvM6Eq8t4Fta72yHwUEUUPefOqlmm'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [22]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereExtractor():
    def __init__(self, examples, example_labels, labels, task_desciption, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_desciption = task_desciption
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_desciption +
                "\n---\n".join( [examples[i] + "\n" +
                                self.example_prompt + 
                                 labels[i] for i in range(len(examples))]))

    def extract(self, example):
      extraction = co.generate(
          model='small',
          prompt=self.make_prompt(example),
          max_tokens=10,
          temperature=0.1,
          stop_sequences=["\n"])
      return(extraction.generations[0].text[:-1])


cohereResearchExtractor = cohereExtractor([e[1] for e in truth_examples], 
                                       [e[0] for e in truth_examples], [],
                                       "", 
                                       "extract key concept from research paper: ")

# Uncomment to inspect the full prompt:
# print(cohereMovieExtractor.make_prompt('<input text here>'))

In [23]:
print(cohereResearchExtractor.make_prompt('<input text here>'))

Some methods for rank correlation in evaluation are examined and their relative advantages and disadvantages are discussed. In particular, it is suggested that different test statistics should be used for providing additional information about the experiments other that the one provided by statistical significance testing. Kendall's τ is often used for testing-rank correlation, yet it is little appropriate if the objective of the test is different from what τ was designed for. In particular, attention should be paid to the null hypothesis. Other measures for rank correlation are described. If one test statistic suggests to reject a hypothesis, other test statistics should be used to support or to revise the decision. The paper then focuses on rank correlation between webpage lists ordered by PageRank for applying the general reflections on these test statistics. An interpretation of PageRank behaviour is provided on the basis of the discussion of the test statistics for rank correlatio

In [30]:
training_set = []
for text in tqdm(abstract_list[:10000]):
    try:
        extracted_text = cohereResearchExtractor.extract(text)
        training_set.append([extracted_text, text])
    except Exception as e:
        print('ERROR: ', e)

  0%|          | 2/10000 [00:00<14:50, 11.22it/s]

ERROR:  You are past your generate endpoint character limit, please request higher tier access!
ERROR:  You are past your generate endpoint character limit, please request higher tier access!
ERROR:  You are past your generate endpoint character limit, please request higher tier access!


  0%|          | 6/10000 [00:00<15:04, 11.05it/s]

ERROR:  You are past your generate endpoint character limit, please request higher tier access!
ERROR:  You are past your generate endpoint character limit, please request higher tier access!
ERROR:  You are past your generate endpoint character limit, please request higher tier access!


  0%|          | 8/10000 [00:00<14:37, 11.39it/s]

...

  1%|          | 82/10000 [00:07<15:23, 10.74it/s]

ERROR:  You are past your generate endpoint character limit, please request higher tier access!
ERROR:  You are past your generate endpoint character limit, please request higher tier access!
ERROR:  You are past your generate endpoint character limit, please request higher tier access!


  1%|          | 84/10000 [00:07<15:14, 10.84it/s]

ERROR:  You are past your generate endpoint character limit, please request higher tier access!


KeyboardInterrupt: 

In [28]:
len(training_set)

0